# GPT-5 mini
## Refined mutation operator generated by GPT-5 mini during Stage 2 of the proposed refinement framework.

In [ ]:
# imports 
import os
import numpy as np
import ioh 
import pandas as pd
import inspect
import importlib
import re 

In [ ]:
# First, we define the DE class in order to use different mutation strategies
class DE:
    """
    Class implementing the Differential Evolution (DE) algorithm in a
    generic manner, allowing the mutation strategy to be easily exchanged.
    """
    def __init__(self,
                 mutation_function,  # Mutation as a function
                 pop_size: int = 50,
                 num_iterations: int = 2000,
                 F: float = 0.8,  # typical value: 0.8
                 CR: float = 0.2):  # typical value: 0.2
        """
        Constructor of the DE algorithm.

        Args:
            mutation_function: Function implementing the mutation strategy.
            pop_size (int): Population size.
            num_iterations (int): Number of generations/iterations.
            F (float): Scaling factor for mutation.
            CR (float): Crossover rate (binomial crossover probability).
        """
        self.mutation_function = mutation_function
        self.pop_size = pop_size
        self.num_iterations = num_iterations
        self.F = F
        self.CR = CR

    def __call__(self, problem: ioh.problem.RealSingleObjective) -> None:
        dim = problem.meta_data.n_variables
        low_b, high_b = problem.bounds.lb[0], problem.bounds.ub[0]

        population = np.random.uniform(low_b, high_b, (self.pop_size, dim))
        fitness = np.array([problem(ind) for ind in population])

        for _ in range(self.num_iterations):
            for i in range(self.pop_size):
                # a. Mutation (now calling the external mutation function)
                mutant = self.mutation_function(
                    i=i,
                    population=population,
                    fitness=fitness,
                    F=self.F,
                    low_b=low_b,
                    high_b=high_b
                )

                # b. Crossover (Binomial) - unchanged
                trial_vector = np.copy(population[i])
                j_rand = np.random.randint(0, dim)
                crossover_mask = np.random.rand(dim) < self.CR
                crossover_mask[j_rand] = True
                trial_vector[crossover_mask] = mutant[crossover_mask]
                
                # c. Selection - unchanged
                trial_fitness = problem(trial_vector)
                
                if trial_fitness <= fitness[i]:
                    population[i] = trial_vector
                    fitness[i] = trial_fitness

In [ ]:
# Mutation functions
# All comments and descriptions within this function were generated
# by the corresponding LLM model during the refinement stage.
def mutation_refined_gpt5mini(i: int, population: np.ndarray, fitness: np.ndarray, F: float, low_b: float, high_b: float) -> np.ndarray:
    """
    Refined variant based on the 'generated_mutation' strategy with the following
    modifications:
    - Uses an F_i sampled from a light-tailed Cauchy distribution (allowing rare jumps),
      followed by clipping.
    - Convexly combines a best-guided (p-best) component and a random (rand) component,
      with adaptive weights based on the rank of the individual.
    - Adds a small adjustment toward/away from the population centroid to balance
      exploration and exploitation.
    - Includes occasional jumps to escape stagnation.
    """
    pop_size, dim = population.shape
    if pop_size == 0:
        return np.clip(np.zeros(dim), low_b, high_b)

    best_idx = int(np.argmin(fitness))

    # --- Adaptive F using a Cauchy distribution (enables rare jumps) with clipping ---
    raw = np.random.standard_cauchy()
    F_i = F + 0.08 * raw
    F_i = float(np.clip(F_i, 0.05, 1.2))

    # --- Normalized rank of individual i (0 = best) ---
    order = np.argsort(fitness)
    ranks = np.empty(pop_size, dtype=int)
    ranks[order] = np.arange(pop_size)
    rank_norm = ranks[i] / max(1, pop_size - 1)  # in [0, 1]

    # Weight favoring exploitation for good individuals and exploration for poor ones
    w_best = 0.2 + 0.7 * (1.0 - rank_norm)  # approx. in [0.2 (worst), 0.9 (best)]

    # --- Safe index selection ---
    candidates = [idx for idx in range(pop_size) if idx != i]
    replace_needed = len(candidates) < 4
    # Select 4 indices to build components (allow replacement if population is small)
    r = (
        np.random.choice(candidates, 4, replace=replace_needed)
        if candidates else [i, i, i, i]
    )
    r1, r2, r3, r4 = int(r[0]), int(r[1]), int(r[2]), int(r[3])

    # --- p-best selection (choose from the top p-fraction) to preserve strong exploitation ---
    p = 0.2
    top_k = max(2, int(np.ceil(p * pop_size)))
    top_indices = [int(idx) for idx in order[:top_k] if int(idx) != i]
    if len(top_indices) == 0:
        pbest_idx = best_idx
    else:
        pbest_idx = int(np.random.choice(top_indices))

    # --- Mutation components ---
    # Best-/p-best-guided component (preserves strengths of generated_mutation)
    best_component = population[pbest_idx] + F_i * (population[r1] - population[r2])

    # Random component (promotes diversity)
    rand_component = population[r3] + F_i * (population[r4] - population[r1])

    # Adaptive convex combination
    mutant = w_best * best_component + (1.0 - w_best) * rand_component

    # --- Centroid-based adjustment to smooth convergence ---
    centroid = np.mean(population, axis=0)
    # For good individuals: small push toward the centroid (refinement).
    # For poor individuals: slight push away from the centroid (encourage exploration).
    centroid_factor = 0.02
    nudge = centroid_factor * (centroid - population[i]) * (1.0 - 2.0 * rank_norm)
    # (if rank_norm < 0.5 -> positive nudge toward centroid; if > 0.5 -> negative, away)
    mutant = mutant + nudge

    # --- Occasional jumps to escape flat regions ---
    if np.random.rand() < 0.06:
        # Jump based on the difference of two random individuals
        cand = [idx for idx in range(pop_size) if idx not in {i, pbest_idx}]
        if len(cand) >= 2:
            rj, rk = np.random.choice(cand, 2, replace=False)
        elif len(cand) == 1:
            rj = cand[0]
            rk = rj
        else:
            rj = rk = i
        jump = 0.6 * F_i * (population[rj] - population[rk])
        mutant = mutant + jump

    # --- Ensure values remain within bounds ---
    return np.clip(mutant, low_b, high_b)

In [ ]:
def analyze_file_list(root_folder, file_list):
    all_results = []

    print(
        f"Analyzing {len(file_list)} files from folder: "
        f"'{os.path.abspath(root_folder)}'"
    )

    for file_name in file_list:
        # Build the full path to the file
        file_path = os.path.join(root_folder, file_name)
        
        try:
            # Extract information from the file name
            match_name = re.match(r"IOHprofiler_f(\d+)_(.*)\.info", file_name)
            if not match_name:
                continue
            
            func_id = int(match_name.group(1))
            func_name = match_name.group(2)

            # Read the file content
            with open(file_path, 'r') as f:
                content = f.read()
            
            # Extract results from repeated runs
            lines = content.splitlines()
            results_line = ""
            for i, current_line in enumerate(lines):
                if current_line.strip() == '%':
                    if i + 1 < len(lines):
                        results_line = lines[i + 1]
                        break
            
            if not results_line:
                raise ValueError(
                    "Results line not found after '%' delimiter"
                )

            matches_data = re.findall(
                r'(\d+)\s*:\s*(\d+)\s*\|\s*([\d.eE+-]+)',
                results_line
            )
            if not matches_data:
                raise ValueError("No result entries were found.")

            # Convert extracted data to numerical lists
            best_fitness_list = [float(m[2]) for m in matches_data]
            evaluations_list = [int(m[1]) for m in matches_data]
            
            # --- COMPUTATIONS ---
            # Compute mean, standard deviation, and median
            avg_fitness = np.mean(best_fitness_list)
            std_fitness = np.std(best_fitness_list)
            median_fitness = np.median(best_fitness_list)
            avg_evals = np.mean(evaluations_list)

            # --- STORE RESULTS ---
            all_results.append({
                "Function_ID": func_id,
                "Function_Name": func_name,
                "Avg_Fitness": avg_fitness,
                "Std_Fitness": std_fitness,
                "Median_Fitness": median_fitness,
                "Avg_Evals": avg_evals
            })
        except FileNotFoundError:
            print(f"- Warning: File '{file_path}' not found")
        except Exception as e:
            print(f"- Error processing file {file_name}: {e}")
    
    if not all_results:
        return None
        
    return pd.DataFrame(all_results)


In [ ]:

de_algorithm = DE(mutation_function=mutation_refined_gpt5mini, pop_size=50, F=0.8, CR=0.2)

experiment = ioh.Experiment(
    algorithm=de_algorithm,
    fids=range(1, 25), #Function range
    iids=[1], #Instances
    dims=[5], #Dimensions
    reps=30, #Number of Repetitions
    zip_output=False,
    folder_name="mutation_refined_gpt5mini",#Name of the output folder where results are stored
    old_logger=True
)

# To run the experiment, simply call it
experiment.run()

In [ ]:
# Analyze and display the results
if __name__ == "__main__":
    ROOT_FOLDER = "mutation_refined_gpt5mini" 

    files_to_analyze = [
        "IOHprofiler_f1_Sphere.info",
        "IOHprofiler_f2_Ellipsoid.info",
        "IOHprofiler_f3_Rastrigin.info",
        "IOHprofiler_f4_BuecheRastrigin.info",
        "IOHprofiler_f5_LinearSlope.info",
        "IOHprofiler_f6_AttractiveSector.info",
        "IOHprofiler_f7_StepEllipsoid.info",
        "IOHprofiler_f8_Rosenbrock.info",
        "IOHprofiler_f9_RosenbrockRotated.info",
        "IOHprofiler_f10_EllipsoidRotated.info",
        "IOHprofiler_f11_Discus.info",
        "IOHprofiler_f12_BentCigar.info",
        "IOHprofiler_f13_SharpRidge.info",
        "IOHprofiler_f14_DifferentPowers.info",
        "IOHprofiler_f15_RastriginRotated.info",
        "IOHprofiler_f16_Weierstrass.info",
        "IOHprofiler_f17_Schaffers10.info",
        "IOHprofiler_f18_Schaffers1000.info",
        "IOHprofiler_f19_GriewankRosenbrock.info",
        "IOHprofiler_f20_Schwefel.info",
        "IOHprofiler_f21_Gallagher101.info",
        "IOHprofiler_f22_Gallagher21.info",
        "IOHprofiler_f23_Katsuura.info",
        "IOHprofiler_f24_LunacekBiRastrigin.info"
    ]

    final_summary_df = analyze_file_list(root_folder=ROOT_FOLDER, file_list=files_to_analyze)

    print(final_summary_df)

In [ ]:
# Save the results to a .csv file
output_csv = "mutation_refined_gpt5mini.csv"
final_summary_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"CSV file saved to: {output_csv}")